In [ ]:
from src.datasets.kitti_dataset import KittiDataset

dataset_path = "dataset/"
sequence = "00"
image_instances_path = "pipeline/vfm-labels/sam/00/"
kitti = KittiDataset(dataset_path, sequence, image_instances_path)

In [ ]:
from src.services.preprocessing.common.config import ConfigDTO

config = ConfigDTO(
    **{
        "dataset": kitti,
        "start_index": 20,
        "end_index": 23,
        "start_image_index_offset": 3,
        "cam_name": "cam2",
        "R": 16,
        "nb_neighbors": 25,
        "std_ratio": 5.0,
        "voxel_size": 0.25,
    }
)

In [ ]:
from src.services.preprocessing.init.map import InitMapProcessor
from src.services.preprocessing.init.instances_matrix import InitInstancesMatrixProcessor

init_pcd = InitMapProcessor().process(config)
points2instances = InitInstancesMatrixProcessor().process(config, init_pcd)

In [ ]:
from src.services.preprocessing.not_zero import SelectionNotZeroProcessor
from src.services.preprocessing.in_cube import SelectionInCubeProcessor
from src.services.preprocessing.statistical_outlier import StatisticalOutlierProcessor

processors = [
    SelectionNotZeroProcessor(),
    SelectionInCubeProcessor(),
    StatisticalOutlierProcessor(),
]

In [ ]:
import copy

pcd = copy.deepcopy(init_pcd)
for processor in processors:
    pcd, points2instances = processor.process(config, pcd, points2instances)

pcd_for_clustering = copy.deepcopy(pcd)

In [ ]:
from src.services.preprocessing.voxel_down import VoxelDownProcessor

pcd, points2instances, trace = VoxelDownProcessor().process(config, pcd, points2instances)

In [ ]:
import numpy as np

from scipy.spatial.distance import cdist
from src.utils.distances_utils import sam_label_distance

points = np.asarray(pcd.points)
spatial_distance = cdist(points, points)

dist, masks = sam_label_distance(points2instances, spatial_distance, 2, 10, 3)

In [ ]:
dist.shape

In [ ]:
from src.utils.distances_utils import remove_isolated_points

dist, points, trace = remove_isolated_points(dist, points, trace)

In [ ]:
dist.shape

In [ ]:
from src.utils.distances_utils import extract_largest_connected_component

dist_final, points_final, trace_final = extract_largest_connected_component(dist, points, trace)

In [ ]:
print(dist_final.shape)
print(len(points_final))
print(len(trace_final))

In [ ]:
dist = dist_final
points = points_final

In [ ]:
from src.services.normalized_cut_service import normalized_cut

T = 0.07
eigenval = 2

clusters = normalized_cut(
    dist,
    np.array([i for i in range(len(points))], dtype=int),
    T,
    eigenval
)

In [ ]:
len(clusters)

In [ ]:
from src.utils.pcd_utils import color_pcd_by_clusters_and_voxels

pcd_colored = color_pcd_by_clusters_and_voxels(pcd_for_clustering, trace_final, clusters)

In [ ]:
from src.utils.pcd_utils import visualize_pcd

visualize_pcd(pcd_colored)